# Note book to create detections over chips for visualzation




In [2]:
import os
from os import makedirs, path as op
from itertools import zip_longest
import json
import base64
import requests

import numpy as np
from skimage import img_as_ubyte
import skimage.io as sio
from skimage.transform import rescale
import click
from tqdm import tqdm

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [3]:
# !pip install numpy scikit-image setuptools -U

In [41]:
def _grouper(iterable, n, fillvalue=None):
    "Itertool recipe to collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)


def _filter_pred_keys(pred_dict, keep_keys=('detection_scores',
                                            'detection_classes',
                                            'detection_boxes')):
    """Apply filter to keep a limited number of dict keys"""
    new_pred_dict = {}
    for key in keep_keys:
        new_pred_dict[key] = pred_dict[key]

    return new_pred_dict


def _apply_score_thresh(pred_dict, score_thresh):
    """Remove predictions below some score threshold"""

    new_pred_dict = {}
    good_inds = np.asarray(pred_dict['detection_scores']) >= score_thresh

    for key in pred_dict.keys():
        new_pred_dict[key] = np.asarray(pred_dict[key])[good_inds].tolist()

    return new_pred_dict

def local_inf(image_directory, url_endpoint, save_directory, score_thresh,
              batch_size):
    """Run inference on local directory using TF Serving image.
    Parameters
    ----------
    image_directory: str
        String specifiying the directory path where all images will be inferred
    url_endpoint: str
        URL of running server (e.g., TF Serving container) where images will
        be sent for inference.
    save_directory: str
        Directory to save predictions. Defaults to same directory as images.
    score_thresh: float
        Minimum score to include a model detection in the final results.
    batch_size: int
        Number of images per inference batch. Defaults to 1.
    img_ext: list of str
        Extensions of files to run prediction on. Defaults to `jpg`.
    """

    if not op.isdir(save_directory):
        makedirs(save_directory)
        
    if not 0. < score_thresh <= 1.:
        raise ValueError('`score_thresh` must fall on interval (0, 1]')

    img_fnames = [img_fname for img_fname in os.listdir(image_directory) if img_fname.endswith('.jpg')]

    # Iterate through groups of images
    for img_group in tqdm(_grouper(img_fnames, batch_size)):
        print(img_group)

        ###################################
        # Create a batch of data to predict

        instances = []
        for batch_img_fname in img_group:
            if batch_img_fname is not None:
                print(batch_img_fname)
                with open(op.join(image_directory, batch_img_fname), 'rb') as image_file:
                    b64_image = base64.b64encode(image_file.read())
                    instances.append({'inputs': {'b64': b64_image.decode('utf-8')}})
                    #instances.append({'inputs': b64_image.decode('utf-8')})

        ################
        # Run prediction
        payload = json.dumps({"instances": instances})
        resp = requests.post(url_endpoint, data=payload)
        print(resp)
        print(resp.content)

#         preds = json.loads(resp.content)['predictions']
#         print(preds)
#         ##########################################
#         # Save results to text file for each image

#         for batch_img_fname, pred_dict in zip(img_group, preds):
#             save_fname = op.splitext(batch_img_fname)[0] + '.json'

#             # Remove some keys, and then apply score threshold
#             pred_dict = _filter_pred_keys(pred_dict)
#             pred_dict = _apply_score_thresh(pred_dict, score_thresh)
#             pred_dict['image_fname'] = batch_img_fname  # Add filename

#             # Save to disk
#             with open(op.join(save_directory, save_fname), 'w') as json_file:
#                 json.dump(pred_dict, json_file)
                
                
def save_annotated_image(json_fpath, cat_pbtxt_fpath, source_directory,
                         save_directory='.', rescale_factor=1.):
    """Save bounding box predictions as annotations on an image"""

    # Load JSON of image predictions and class indicies
    jsons = [op.join(json_fpath, json_) for json_ in os.listdir(json_fpath)]
    for json_ in jsons: 
        with open(json_, 'r') as json_file:
            pred = json.load(json_file)

        cat_index = label_map_util.create_category_index_from_labelmap(
            cat_pbtxt_fpath, use_display_name=True)

        # Generate file paths to save/load image
        img_fname = op.splitext(op.basename(json_))[0] + '.jpg'
        img_load_fpath = op.join(source_directory, img_fname)
        img_save_fpath = op.join(save_directory, img_fname)

        # Load image and rescale if necessary
        image_np = sio.imread(img_load_fpath)
        if rescale_factor != 1.:
            image_np = rescale(image_np, anti_aliasing=True, multichannel=True,
                               scale=rescale_factor, clip=False)
    #         image_np = img_as_ubyte(image_np)

        # Increase font size
        # Move arial.ttf to /models/research/object_detection/utils
        #font = ImageFont.truetype('./arial.ttc', 36)

        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            np.asarray(pred['detection_boxes']),
            np.asarray(pred['detection_classes'], dtype=np.uint8),
            np.asarray(pred['detection_scores']),
            cat_index,
            instance_masks=pred.get('detection_masks'),
            use_normalized_coordinates=True,
            line_thickness=1)

        # Save output; time-consuming for large images
        sio.imsave(img_save_fpath, image_np)

In [43]:
image_directory = 'data/cormon2019_chips'

#Get ip addrees docker network inspect ai4earth-wildlife-conservation_default

server_endpoint = 'http://172.23.0.1:8501/v1/models/wildlife:predict'
# r = requests.get(server_endpoint)
# print(r)

score_thresh = 0.5
batch_size = 5
directory = "data/cormon2019_chips_local_inf"
local_inf(image_directory, server_endpoint, directory, score_thresh,batch_size)



2it [00:00, 18.12it/s]

('wcm_n51_L_20190602095310_0_0.jpg', 'wcm_n51_L_20190602095310_0_2.jpg', 'wcm_n51_L_20190602095310_0_3.jpg', 'wcm_n51_L_20190602095310_0_4.jpg', 'wcm_n51_L_20190602095310_0_5.jpg')
wcm_n51_L_20190602095310_0_0.jpg
wcm_n51_L_20190602095310_0_2.jpg
wcm_n51_L_20190602095310_0_3.jpg
wcm_n51_L_20190602095310_0_4.jpg
wcm_n51_L_20190602095310_0_5.jpg
<Response [400]>
b'{ "error": "Attempting to use uninitialized value FirstStageFeatureExtractor/resnet_v1_101/block3/unit_12/bottleneck_v1/conv3/weights\\n\\t [[{{node FirstStageFeatureExtractor/resnet_v1_101/block3/unit_12/bottleneck_v1/conv3/weights/read}}]]" }'
('wcm_n51_L_20190602095310_0_6.jpg', 'wcm_n51_L_20190602095310_0_7.jpg', 'wcm_n51_L_20190602095310_0_8.jpg', 'wcm_n51_L_20190602095310_0_9.jpg', 'wcm_n51_L_20190602095310_10_0.jpg')
wcm_n51_L_20190602095310_0_6.jpg
wcm_n51_L_20190602095310_0_7.jpg
wcm_n51_L_20190602095310_0_8.jpg
wcm_n51_L_20190602095310_0_9.jpg
wcm_n51_L_20190602095310_10_0.jpg
<Response [400]>
b'{ "error": "Attempting

11it [00:00, 27.25it/s]

<Response [400]>
b'{ "error": "Attempting to use uninitialized value FirstStageFeatureExtractor/resnet_v1_101/block3/unit_12/bottleneck_v1/conv3/weights\\n\\t [[{{node FirstStageFeatureExtractor/resnet_v1_101/block3/unit_12/bottleneck_v1/conv3/weights/read}}]]" }'
('wcm_n51_L_20190602095310_11_6.jpg', 'wcm_n51_L_20190602095310_11_7.jpg', 'wcm_n51_L_20190602095310_11_8.jpg', 'wcm_n51_L_20190602095310_11_9.jpg', 'wcm_n51_L_20190602095310_12_0.jpg')
wcm_n51_L_20190602095310_11_6.jpg
wcm_n51_L_20190602095310_11_7.jpg
wcm_n51_L_20190602095310_11_8.jpg
wcm_n51_L_20190602095310_11_9.jpg
wcm_n51_L_20190602095310_12_0.jpg
<Response [400]>
b'{ "error": "Attempting to use uninitialized value FirstStageFeatureExtractor/resnet_v1_101/block3/unit_12/bottleneck_v1/conv3/weights\\n\\t [[{{node FirstStageFeatureExtractor/resnet_v1_101/block3/unit_12/bottleneck_v1/conv3/weights/read}}]]" }'
('wcm_n51_L_20190602095310_12_1.jpg', 'wcm_n51_L_20190602095310_12_2.jpg', 'wcm_n51_L_20190602095310_12_3.jpg', 'w

In [52]:
# #building parts 
# json_fpath = "building_parts_dir_local_inf"
# cat_pbtxt_fpath = 'building_parts.pbtxt'
# source_directory = "gif_creation_local_inference"
# save_directory = json_fpath
# save_annotated_image(json_fpath, cat_pbtxt_fpath, source_directory,
#                          save_directory, rescale_factor=1.)

In [56]:
# # building properties
# json_fpath = "building_properties_dir_local_inf"
# cat_pbtxt_fpath = 'building_properties-padang.pbtxt'
# source_directory = "gif_creation_local_inference"
# save_directory = json_fpath
# save_annotated_image(json_fpath, cat_pbtxt_fpath, source_directory,
#                          save_directory, rescale_factor=1.)